In [ ]:

---

# 🟦 **fraud_risk_engine.py**

This is a **complete end-to-end fraud detection & risk scoring system**.

```python
# ================================================
# AI-DRIVEN FRAUD RISK SCORING ENGINE
# ================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# -------------------------------
# 1. Load Dataset
# -------------------------------
# Dataset must have a column "Fraud" (1 = fraud, 0 = normal)
df = pd.read_csv("transactions.csv")
print("Dataset Loaded")

# -------------------------------
# 2. Data Cleaning
# -------------------------------
df = df.dropna()

# -------------------------------
# 3. Feature & Target Split
# -------------------------------
X = df.drop("Fraud", axis=1)
y = df["Fraud"]

# -------------------------------
# 4. Train-Test Split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# -------------------------------
# 5. Feature Scaling
# -------------------------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# -------------------------------
# 6. Logistic Regression Model
# -------------------------------
log_model = LogisticRegression(class_weight="balanced")
log_model.fit(X_train, y_train)

# -------------------------------
# 7. Random Forest Model
# -------------------------------
rf_model = RandomForestClassifier(n_estimators=300, class_weight="balanced", random_state=42)
rf_model.fit(X_train, y_train)

# -------------------------------
# 8. Model Evaluation
# -------------------------------
rf_pred = rf_model.predict(X_test)
rf_prob = rf_model.predict_proba(X_test)[:,1]

print("\nRandom Forest Fraud Detection Report:")
print(classification_report(y_test, rf_pred))

roc = roc_auc_score(y_test, rf_prob)
print("ROC-AUC Score:", roc)

# -------------------------------
# 9. Confusion Matrix
# -------------------------------
cm = confusion_matrix(y_test, rf_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Reds")
plt.title("Fraud Detection Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# -------------------------------
# 10. ROC Curve
# -------------------------------
fpr, tpr, _ = roc_curve(y_test, rf_prob)

plt.plot(fpr, tpr, label=f"AUC = {roc:.2f}")
plt.plot([0,1],[0,1],'--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Fraud Detection ROC Curve")
plt.legend()
plt.show()

# -------------------------------
# 11. Generate Fraud Risk Scores
# -------------------------------
df["Fraud_Risk_Score"] = rf_model.predict_proba(scaler.transform(X))[:,1]

# -------------------------------
# 12. Save High-Risk Transactions
# -------------------------------
high_risk = df[df["Fraud_Risk_Score"] > 0.7]
high_risk.to_csv("high_risk_transactions.csv", index=False)

print("\nHigh-risk transactions saved successfully!")
